In [ ]:
 from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy

def load_kernel_matrix_from_splits(base_path, dataset_size, size_splits, number_splits, file_name_start="NTK_"):

  l = lambda x : x*size_splits 
  h = lambda x : (1+x)*size_splits if (not x == number_splits) else dataset_size-1

  kernel_matrix = np.empty((dataset_size, dataset_size))

  for i in range(number_splits+1):
    for j in range(number_splits+1):    
      split_k = np.load(kernel_path + f"/NTK_{h(i)}_{h(j)}.npy")
      kernel_matrix[l(i):h(i),l(j):h(j)] = split_k
    
  return kernel_matrix


def kernel_SVM(kernel_matrix, ys):
  C_list = np.logspace(-2, 4, 120)
  svc = SVC(kernel='precomputed', max_iter=5000)
  clf = GridSearchCV(svc, {'C' : C_list}, verbose=0, return_train_score=True)
  clf.fit(kernel_matrix, ys)

  return clf, pd.DataFrame(clf.cv_results_)

In [ ]:
# GCNN data
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/GCNN"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 50, 3)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

In [ ]:
kernel_matrix[-5:,-5:]

array([[  73008.,   72072.,  393120.,  393120.,   88920.],
       [  72072.,   71148.,  388080.,  388080.,   87780.],
       [ 393120.,  388080., 2116800., 2116800.,  478800.],
       [ 393120.,  388080., 2116800., 2116800.,  478800.],
       [  88920.,   87780.,  478800.,  478800.,  108300.]])

In [ ]:
# estimate kernel svm
svc, result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
39,0.00502,0.000183,0.001222,0.000044,0.925522,{'C': 0.9255215909791353},0.921053,0.921053,0.837838,0.864865,...,0.876529,0.037669,1,0.845638,0.818792,0.866667,0.86,0.866667,0.851553,0.018091


In [ ]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np

krr = KernelRidge(alpha=1.0, kernel="precomputed")
krr.fit(kernel_matrix, ys)

np.sum(np.array(krr.predict(kernel_matrix) < 0.5, dtype="int") == ys) / 188

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [ ]:
svc = SVC(kernel='precomputed', max_iter=5e5, C=6285.203269)
svc.fit(kernel_matrix, ys)
svc.predict(kernel_matrix)

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1], dtype=int32)

In [ ]:
# Two_wl Kernel
twl_radius = 4
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/TWO_WL/radius_{twl_radius}"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 15, 12)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

# estimate kernel svm
svc, result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
109,0.004025,0.000424,0.000968,0.000114,3131.831005,{'C': 3131.8310052438446},0.684211,0.789474,0.783784,0.675676,...,0.743385,0.051912,1,0.724832,0.832215,0.646667,0.666667,0.733333,0.720743,0.064827
